# BlazePalm

In [1]:
import os
import glob
import math

import cv2
import pandas as pd
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

import tflite

from nets import blaze_palm

## Reproduce BlazePalm Model Structure

In [2]:
model = blaze_palm.build_blaze_palm_model()

## Get Weights From Mediapipe's BlazePalm

In [3]:
buf = open('original_models/palm_detection.tflite', 'rb').read()
rel_model = tflite.Model.GetRootAsModel(buf, 0)
subgraph = rel_model.Subgraphs(0)

weights_dict = {}
weights_shape_dict = {}
rel_model_names = []

for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    name = tensor.Name().decode()
    buffer_idx = tensor.Buffer()
    buffer = rel_model.Buffers(buffer_idx)

    w = buffer.DataAsNumpy()
    if type(w) is np.ndarray:
        w = np.frombuffer(w, dtype=np.float32)
    weights_dict[name] = w
    rel_model_names.append(name)
    weights_shape_dict[name] = tensor.ShapeAsNumpy()

## Get Layers From Reproduced BlazePalm

In [4]:
layer_name_list = []
layer_shape_list = []
layer_name_layer_map = {}
for layer in model.layers:
    if "conv2d" in layer.name:
        layer_name_list.append(layer.name + '/Kernel')
        layer_name_list.append(layer.name + '/Bias')
        layer_name_list.append(layer.name)
        layer_shape_list.append(layer.get_weights()[0].shape)
        layer_shape_list.append(layer.get_weights()[1].shape)
        layer_shape_list.append(layer.output_shape)
        layer_name_layer_map[layer.name] = layer
    elif "pad" in layer.name:
        layer_name_list.append(layer.name + '/Paddings')
        layer_name_list.append(layer.name)
        layer_shape_list.append(layer.output_shape)
        layer_shape_list.append(layer.output_shape)
        layer_name_layer_map[layer.name] = layer
    else:
        layer_name_list.append(layer.name)
        layer_shape_list.append(layer.output_shape)


## Find Layers That Do Not Match

In [5]:
no_direct_match_model_name = [layer_name for layer_name in layer_name_list if layer_name not in weights_dict]
no_direct_match_rel_model_name = [layer_name for layer_name in weights_dict if layer_name not in layer_name_list]

In [6]:
for i in range(len(rel_model_names)):
    print(
        rel_model_names[i].ljust(50), 
        str(weights_shape_dict[rel_model_names[i]]).ljust(20), 
        layer_name_list[i].ljust(50), 
        str(layer_shape_list[i]).ljust(20), 
        rel_model_names[i] in no_direct_match_rel_model_name,  
        layer_name_list[i] in no_direct_match_model_name
        )

input                                              [  1 256 256   3]    input_1                                            [(1, 256, 256, 3)]   True True
conv2d/Kernel                                      [32  3  3  3]        conv2d/Kernel                                      (3, 3, 3, 32)        False False
conv2d/Bias                                        [32]                 conv2d/Bias                                        (32,)                False False
conv2d                                             [  1 128 128  32]    conv2d                                             (1, 128, 128, 32)    False False
activation                                         [  1 128 128  32]    activation                                         (1, 128, 128, 32)    False False
depthwise_conv2d/Kernel                            [ 1  3  3 32]        depthwise_conv2d/Kernel                            (3, 3, 32, 1)        False False
depthwise_conv2d/Bias                              [32]           

In [7]:
no_direct_match_mapping = {
    "conv2d_42": "regressor_8",
    "conv2d_43": "regressor_16",
    "conv2d_44": "regressor_32",
    "conv2d_45": "classificator_8",
    "conv2d_46": "classificator_16",
    "conv2d_47": "classificator_32",
}

## Copy Over the Weights

In [8]:
for n in layer_name_layer_map.keys():
    if "conv2d" not in n: # Only conv2d has weights to transfer. SSD is fully conv
        continue

    layer = layer_name_layer_map[n]
    
    if n in no_direct_match_mapping:
        n = no_direct_match_mapping[n]

    kernel_name = n + '/Kernel'
    bias_name = n + '/Bias'

    assert kernel_name in weights_dict
    assert bias_name in weights_dict

    kernel_weights = weights_dict[kernel_name]
    kernel_weights = kernel_weights.reshape(weights_shape_dict[kernel_name])
    if ("transpose" not in kernel_name):
        kernel_weights = kernel_weights.transpose((1, 2, 3, 0))
    else:
        kernel_weights = kernel_weights.transpose((1, 2, 0, 3))

    bias_weights = weights_dict[bias_name]
    
    # Copy weights over
    print(
        n.ljust(30),
        str(weights_shape_dict[kernel_name]).ljust(20),
        str(kernel_weights.shape).ljust(10),
        str(layer.get_weights()[0].shape).ljust(40),
        str(weights_shape_dict[bias_name]).ljust(10),
        str(bias_weights.shape).ljust(10),
        str(layer.get_weights()[1].shape).ljust(10),
    )

    assert kernel_weights.shape == layer.get_weights()[0].shape
    assert bias_weights.shape == layer.get_weights()[1].shape
    layer.set_weights([kernel_weights, bias_weights])


conv2d                         [32  3  3  3]        (3, 3, 3, 32) (3, 3, 3, 32)                            [32]       (32,)      (32,)     
depthwise_conv2d               [ 1  3  3 32]        (3, 3, 32, 1) (3, 3, 32, 1)                            [32]       (32,)      (32,)     
conv2d_1                       [32  1  1 32]        (1, 1, 32, 32) (1, 1, 32, 32)                           [32]       (32,)      (32,)     
depthwise_conv2d_1             [ 1  3  3 32]        (3, 3, 32, 1) (3, 3, 32, 1)                            [32]       (32,)      (32,)     
conv2d_2                       [32  1  1 32]        (1, 1, 32, 32) (1, 1, 32, 32)                           [32]       (32,)      (32,)     
depthwise_conv2d_2             [ 1  3  3 32]        (3, 3, 32, 1) (3, 3, 32, 1)                            [32]       (32,)      (32,)     
conv2d_3                       [32  1  1 32]        (1, 1, 32, 32) (1, 1, 32, 32)                           [32]       (32,)      (32,)     
depthwise_conv2d_

## Save Model

In [9]:
tf.saved_model.save(model, "palm_detection_tf_finetuned")

INFO:tensorflow:Assets written to: palm_detection_tf_finetuned\assets


In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model("palm_detection_tf_finetuned")

tflite_model = converter.convert()

TFLITE_MODEL_PATH = '../../models/palm_detection_tf_finetuned.tflite'
with open(TFLITE_MODEL_PATH, 'wb') as f:
  f.write(tflite_model)


## Verify TFLite Model

In [11]:
release_model = tf.lite.Interpreter(model_path=TFLITE_MODEL_PATH)

In [12]:
release_model.get_input_details()

[{'name': 'serving_default_input_1:0',
  'index': 0,
  'shape': array([  1, 256, 256,   3]),
  'shape_signature': array([  1, 256, 256,   3]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [13]:
release_model.get_output_details()

[{'name': 'StatefulPartitionedCall:0',
  'index': 300,
  'shape': array([   1, 2944,   18]),
  'shape_signature': array([   1, 2944,   18]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'StatefulPartitionedCall:1',
  'index': 303,
  'shape': array([   1, 2944,    1]),
  'shape_signature': array([   1, 2944,    1]),
  'dtype': numpy.float32,
  'quantization': (0.0, 0),
  'quantization_parameters': {'scales': array([], dtype=float32),
   'zero_points': array([], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]